In [110]:
import spacy
import json
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from spacy.training.example import Example

nlp = spacy.load("en_core_web_sm")

In [12]:
 def load_mappings(dataset):
    f = open(dataset, 'r',encoding='utf-8')
    model= dict({})
    for line in f:
        tem=line.split()
        vtem=[]
        for i in range(1,len(tem)):
            vtem.append( float(tem[i].strip()))
        vec1=np.array(vtem)
        model.update({tem[0].strip():vec1})
    return model

In [133]:
my_text = "I want a blue smartwatch with heart rate monitoring and sleep alarms."
doc = nlp(my_text)

In [134]:
for token in doc:
    if ("dobj" in token.dep_):
        subtree = list(token.subtree)
        start = subtree[0].i
        end = subtree[-1].i + 1
        # return subtree(list), token(product), extract(string)
        extract = doc[start:end]
        product = token.text


In [135]:
product

'smartwatch'

In [136]:
extract

a blue smartwatch with heart rate monitoring and sleep alarms

In [17]:
#load w2v
load_model = load_mappings("glove.6B.300d.txt")

In [70]:
# get_category 
def get_category(product, my_list):
    if product in load_model:
        embeddings = []
        v1 = load_model[product.strip()]
        embeddings.append(v1)
        for category in my_list:
            embeddings.append(load_model[category.strip()])
        cos_sim = cosine_similarity([embeddings[0]], embeddings[1:])
        return my_list[np.argmax(cos_sim)]
    return 0

In [57]:
category_list = ["phone", "laptop","clothes"]

In [137]:
my_category = get_category(token.text, category_list)
my_category

'phone'

In [138]:
# Sau khi có category thì đi vào file json để lấy ra các danh mục của item attributes
with open('data/category-list.json') as f:
    data = json.load(f)

# Lấy ra tất cả các category
phones = [item for item in data if item['category'] == my_category]
for item in phones:
    attributes_json = item['item_attributes']

list_attributes_json = list(attributes_json.keys())
# Cần đưa về chữ thường để tránh các trường hợp chữ hoa W2V không bắt được
list_attributes_json = [x.lower() for x in list_attributes_json]

dict_attributes_json = dict.fromkeys(list_attributes_json, None)

print(list_attributes_json)
print(dict_attributes_json)

['color', 'ram', 'price']
{'color': None, 'ram': None, 'price': None}


In [139]:
# Bước 3
print(extract.text)
doc = nlp(extract.text)
list_attributes_noun = []
list_attributes_adj = []
list_attributes_num = []
# lấy ra các noun, adj và num không phải là product lưu vào 3 list
for token in doc:
    if token.pos_ == "ADJ":
        list_attributes_adj.append(token.text.lower())
    elif token.pos_ == "NOUN" and token.text != str(product):
        list_attributes_noun.append(token.text.lower())
    elif token.pos_ == "NUM":
        list_attributes_num.append(token.text)

a blue smartwatch with heart rate monitoring and sleep alarms


In [140]:
print(list_attributes_json)

print(list_attributes_noun)
print(list_attributes_adj)
print(list_attributes_num)

['color', 'ram', 'price']
['heart', 'rate', 'monitoring', 'sleep', 'alarms']
['blue']
[]


In [141]:
# lấy ra các attributes trong context và gắn vào dict
def get_attributes_list(list_attributes_json, list_attributes_adj, list_attributes_num):
    dict_attributes_json = dict.fromkeys(list_attributes_json, None)
    embeddings_json = []
    
    # lấy ra vecto của các từ trong list_attributes_json
    for attributes_json in list_attributes_json: 
        if attributes_json in load_model:
            v1 = load_model[attributes_json.strip()]
            embeddings_json.append(v1)
            
    # lấy ra vecto của các từ trong list_attributes_adj
    for attributes_adj in list_attributes_adj:
        embeddings_adj = []
        if attributes_adj in load_model:
            v1 = load_model[attributes_adj.strip()]
            embeddings_adj.append(v1)
            cos_sim = cosine_similarity( [embeddings_adj[0]], embeddings_json)
            dict_attributes_json[list_attributes_json[np.argmax(cos_sim)]] = attributes_adj
    
    return dict_attributes_json

In [142]:
similar_attributes = get_attributes_list(list_attributes_json, list_attributes_adj, list_attributes_num)
similar_attributes

{'color': 'blue', 'ram': None, 'price': None}

In [80]:
text = "a blue phone and 8G ram"

# Process the text
doc = nlp(text)

for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

a           DET       det       
blue        ADJ       amod      
phone       NOUN      nsubj     
and         CCONJ     cc        
8           NUM       nummod    
G           NOUN      nsubj     
ram         VERB      ROOT      
